In [1]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('teste_00.db')

# Create a cursor object
cursor = conn.cursor()

# Get the list of tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables = cursor.fetchall()

# Iterate over the tables and retrieve their schema
for table in tables:
    table_name = table[0]
    cursor.execute(f"PRAGMA table_info({table_name})")
    columns = cursor.fetchall()

    print(f"Table: {table_name}")
    for column in columns:
        column_name = column[1]
        data_type = column[2]
        is_nullable = 'NULL' if column[3] == 1 else 'NOT NULL'
        is_primary_key = 'PRIMARY KEY' if column[5] == 1 else ''
        
        print(f"- {column_name} {data_type} {is_nullable} {is_primary_key}")

    print()

# Close the cursor and the connection
cursor.close()
conn.close()

Table: projetos
- n_projeto TEXT NOT NULL 
- id_projeto INTEGER NOT NULL PRIMARY KEY

Table: sqlite_sequence
- name  NOT NULL 
- seq  NOT NULL 

Table: registros
- id_registro INTEGER NOT NULL PRIMARY KEY
- id_projeto INTEGER NOT NULL 
- data_status REAL NOT NULL 
- tipo TEXT NOT NULL 

Table: disciplinas
- id_disciplina INTEGER NOT NULL PRIMARY KEY
- disciplina TEXT NOT NULL 

Table: activities
- id_activity INTEGER NOT NULL PRIMARY KEY
- activity TEXT NOT NULL 
- indice INTEGER NOT NULL 
- unidade TEXT NOT NULL 
- id_disciplina INTEGER NOT NULL 

Table: documentos
- n_poyry TEXT NOT NULL 
- n_cliente TEXT NOT NULL PRIMARY KEY
- avanco REAL NOT NULL 
- a1eq_bl REAL NOT NULL 
- a1eq_rp REAL NOT NULL 
- id_disciplina INTEGER NOT NULL 
- id_registro INTEGER NOT NULL 

Table: custos
- id_custo INTEGER NOT NULL PRIMARY KEY
- categoria TEXT NOT NULL 
- base_cost REAL NOT NULL 
- quantidade REAL NOT NULL 
- id_activity INTEGER NOT NULL 
- id_registro INTEGER NOT NULL 



In [11]:
# Connect to the SQLite database
conn = sqlite3.connect('teste_00.db')

activity_ids = {}
# Create a cursor object
cursor = conn.cursor()


# Retrieve the inserted registro ID
registro_id = cursor.lastrowid

# Populate custos table
custos_data = [
    ('Categoria 1', 10.5, 3.2, 'B - ENGINEERING SERVICES', registro_id),
    ('Categoria 2', 8.7, 2.1, 'D - PRODUCT DEVELOPMENT', registro_id),
    # Add more data as needed
]

for row in cursor.execute("SELECT id_activity, activity FROM activities"):
    activity_ids[row[1]] = row[0]
    
custos_data_with_ids = []
for custo in custos_data:
    categoria, base_cost, quantidade, activity_name, registro_id = custo
    id_activity = activity_ids.get(activity_name)
    if id_activity is not None:
        custos_data_with_ids.append((categoria, base_cost, quantidade, id_activity, registro_id))


In [12]:
activity_ids

{'A - PROJECT MANAGEMENT': 22,
 'B - CIVIL - METÁLICA': 8,
 'B - ENGINEERING SERVICES': 9,
 'C - CIVIL - CONCRETO': 10,
 'C - CIVIL ENGINEERING': 11,
 'D - PRODUCT DEVELOPMENT': 21,
 'E - ELECTRICAL ENGINEERING': 12,
 'External Time, billable': 30,
 'F - CONSTR. MANAGEMENT SERVICES': 16,
 'G - STUDY AND RESEARCH SERVICES': 18,
 'H - PROCUREMENT': 19,
 'I - INSTRUMENT / PROCESS CONTROL ENGINEERING': 13,
 'J - AUTOMATION SYSTEM ENGINEERING': 14,
 'JD - DIGITALISATION': 17,
 'K - INVESTMENT COST ESTIM. & CONTROL': 20,
 'L - ARCHITECTURAL DESIGN': 6,
 'L - CIVIL - ARQUITETURA': 5,
 'M - MECHANICAL ENGINEERING': 3,
 'Miscellaneous costs': 24,
 'P - PROCESS ENGINEERING': 1,
 'R - ADP SERVICES': 15,
 'Revenue evaluation': 25,
 'S - PROJECT PLANNING & FOLLOW-UP': 23,
 'Subconsulting,  other expenses external': 28,
 'T - PIPING ENGINEERING': 4,
 'Travel costs': 26,
 'U - TRAINING SERVICES': 27,
 'V - HVAC & SANITARY SYSTEM DESIGN': 2,
 'W - INFRA SERVICES': 7,
 'Y - HEALTH & SAFETY': 29}

In [13]:
custos_data_with_ids

[('Categoria 1', 10.5, 3.2, 9, None), ('Categoria 2', 8.7, 2.1, 21, None)]

In [6]:
import sqlite3
from datetime import date

# Connect to the SQLite database
conn = sqlite3.connect('teste_00.db')

# Create a cursor object
cursor = conn.cursor()

# Define the data to be inserted
id_projeto = 1
tipo = 'CUSTO'
today = date.today()

# Check if id_projeto already exists in projetos table
cursor.execute("SELECT COUNT(*) FROM projetos WHERE id_projeto = ?", (id_projeto,))
projeto_exists = cursor.fetchone()[0] > 0

# Check if tipo, data_status, and id_projeto already exist in registros table
cursor.execute("SELECT COUNT(*) FROM registros WHERE tipo = ? AND data_status = ? AND id_projeto = ?", (tipo, today, id_projeto))
registro_exists = cursor.fetchone()[0] > 0

# If projeto or registro already exists, display an error message and stop execution
if projeto_exists:
    print(f"Error: id_projeto {id_projeto} already exists in the projetos table.")

if registro_exists:
    print(f"Error: Registro with tipo '{tipo}', data_status '{today}', and id_projeto {id_projeto} already exists in the registros table.")

# Insert data into projetos table
if not projeto_exists:
    cursor.execute("INSERT INTO projetos (id_projeto) VALUES (?)", (id_projeto,))

# Insert data into registros table
if not registro_exists:
    cursor.execute("INSERT INTO registros (id_projeto, data_status, tipo) VALUES (?, ?, ?)", (id_projeto, today, tipo))

# Retrieve the inserted registro ID
registro_id = cursor.lastrowid

# Populate custos table
custos_data = [
    ('Categoria 1', 10.5, 3.2, 'Travel costs', registro_id),
    ('Categoria 2', 8.7, 2.1, 'Travel costs', registro_id),
    # Add more data as needed
]

# Retrieve activity IDs based on activity names
activity_ids = {}
for row in cursor.execute("SELECT id_activity, activity FROM activities"):
    activity_ids[row[1]] = row[0]

# Prepare custos data with activity IDs
custos_data_with_ids = []
for custo in custos_data:
    categoria, base_cost, quantidade, activity_name, registro_id = custo
    id_activity = activity_ids.get(activity_name)
    if id_activity is not None:
        custos_data_with_ids.append((categoria, base_cost, quantidade, id_activity, registro_id))

# Insert custos data into the table
cursor.executemany("INSERT INTO custos (categoria, base_cost, quantidade, id_activity, id_registro) VALUES (?, ?, ?, ?, ?)", custos_data_with_ids)

# Commit the changes to the database
conn.commit()

# Close the cursor and the connection
cursor.close()
conn.close()

Error: id_projeto 1 already exists in the projetos table.
Error: Registro with tipo 'CUSTO', data_status '2023-07-05', and id_projeto 1 already exists in the registros table.


In [8]:
# Connect to the SQLite database
conn = sqlite3.connect('teste_00.db')

# Create a cursor object
cursor = conn.cursor()

cursor.execute("SELECT * FROM custos")
print(cursor.fetchall())

# Close the cursor and the connection
cursor.close()
conn.close()

[(1, 'ET05', 169.11, 480.0, 16, 2), (2, 'ET04', 120.21, 620.0, 16, 2), (3, 'ET03', 91.87, 600.0, 16, 2), (4, 'ET04', 113.59, 84.0, 15, 2), (5, 'ET03', 99.5, 378.0, 15, 2), (6, 'ET02', 77.98, 378.0, 15, 2), (7, 'ET03', 117.04, 350.0, 20, 2), (8, 'ET04', 126.7, 350.0, 20, 2), (9, 'ET04', 120.21, 400.0, 19, 2), (10, 'ET06', 224.54, 104.0, 1, 2), (11, 'ET05', 169.11, 185.25, 1, 2), (12, 'ET04', 120.21, 934.0, 1, 2), (13, 'ET03', 91.87, 813.0, 1, 2), (14, 'ET02', 62.9, 84.0, 1, 2), (15, 'ET01', 36.31, 44.12913905744057, 1, 2), (16, 'ET05', 169.11, 27.25, 14, 2), (17, 'ET04', 120.21, 1453.0, 14, 2), (18, 'ET03', 95.63, 1678.0, 14, 2), (19, 'ET02', 60.7, 597.0, 14, 2), (20, 'ET01', 32.4, 366.58932135975647, 14, 2), (21, 'ET05', 169.11, 0.0, 7, 2), (22, 'ET04', 136.13, 40.0, 7, 2), (23, 'ET03', 95.7, 44.0, 7, 2), (24, 'ET02', 52.13, 0.0, 7, 2), (25, 'ET01', 32.63, 0.0, 7, 2), (26, 'ET05', 169.11, 16.25, 5, 2), (27, 'ET04', 136.13, 1075.0, 5, 2), (28, 'ET03', 95.7, 930.0, 5, 2), (29, 'ET02', 52

In [2]:
import sqlite3 as sql

In [3]:
con = sql.connect('teste_00.db')
cur = con.cursor()

In [14]:
cur.execute("SELECT * FROM categorias")
cur.fetchall()

[(1, 'ET05', 16),
 (2, 'ET04', 16),
 (3, 'ET03', 16),
 (4, 'ET04', 15),
 (5, 'ET03', 15),
 (6, 'ET02', 15),
 (7, 'ET03', 20),
 (8, 'ET04', 20),
 (9, 'ET04', 19),
 (10, 'ET06', 1),
 (11, 'ET05', 1),
 (12, 'ET04', 1),
 (13, 'ET03', 1),
 (14, 'ET02', 1),
 (15, 'ET01', 1),
 (16, 'ET05', 14),
 (17, 'ET04', 14),
 (18, 'ET03', 14),
 (19, 'ET02', 14),
 (20, 'ET01', 14),
 (21, 'ET05', 7),
 (22, 'ET04', 7),
 (23, 'ET03', 7),
 (24, 'ET02', 7),
 (25, 'ET01', 7),
 (26, 'ET05', 5),
 (27, 'ET04', 5),
 (28, 'ET03', 5),
 (29, 'ET02', 5),
 (30, 'ET01', 5),
 (31, 'ET05', 10),
 (32, 'ET04', 10),
 (33, 'ET03', 10),
 (34, 'ET02', 10),
 (35, 'ET01', 10),
 (36, 'ET05', 8),
 (37, 'ET04', 8),
 (38, 'ET03', 8),
 (39, 'ET02', 8),
 (40, 'ET01', 8),
 (41, 'ET06', 12),
 (42, 'ET05', 12),
 (43, 'ET04', 12),
 (44, 'ET03', 12),
 (45, 'ET02', 12),
 (46, 'ET01', 12),
 (47, 'ET05', 3),
 (48, 'ET04', 3),
 (49, 'ET03', 3),
 (50, 'ET02', 3),
 (51, 'ET01', 3),
 (52, 'ET05', 4),
 (53, 'ET04', 4),
 (54, 'ET03', 4),
 (55, 'ET02'

In [4]:
cur.execute(f"PRAGMA table_info(registros)")
columns = cur.fetchall()
columns

[(0, 'id_registro', 'INTEGER', 0, None, 1),
 (1, 'id_projeto', 'INTEGER', 0, None, 0),
 (2, 'data_status', 'REAL', 0, None, 0),
 (3, 'tipo', 'TEXT', 0, None, 0)]

In [3]:
import sqlite3 as sql

con = sql.connect('teste_00.db')
cur = con.cursor()

cur.execute("SELECT * FROM registros")
cur.fetchall()

[(1, 1, '2023-06-21', 'AS SOLD'),
 (2, 1, '2023-06-23', 'MACONOMY'),
 (3, 1, '2023-06-14', 'LISTA DE DESENHOS E DOCUMENTOS'),
 (4, 4, '2023-05-16', 'LISTA DE DESENHOS E DOCUMENTOS')]

In [5]:
con.commit()
con.close()